![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

### Kalman Filters and Pairs Trading
    
There are a few Python packages out there for Kalman filters, but we're adapting this example and the Kalman filter class code from [this article](https://www.quantstart.com/articles/kalman-filter-based-pairs-trading-strategy-in-qstrader) and demonstrating how you can implement similar ideas using QuantConnect!,
    
Briefly, a Kalman filter is a [state-space model](https://en.wikipedia.org/wiki/State-space_representation) applicable to linear dynamic systems -- systems whose state is time-dependent and state variations are represented linearly. The model is used to estimate unknown states of a variable based on a series of past values. The procedure is two-fold: a prediction (estimate) is made by the filter of the current state of a variable and the uncertainty of the estimate itself. When new data is available, these estimates are updated. There is a lot of information available about Kalman filters, and the variety of their applications is pretty astounding, but for now, we're going to use a Kalman filter to estimate the hedge ratio between a pair of equities.
   
The idea behind the strategy is pretty straightforward: take two equities that are cointegrated and create a long-short portfolio. The premise of this is that the spread between the value of our two positions should be mean-reverting. Anytime the spread deviates from its expected value, one of the assets moved in an unexpected direction and is due to revert back. When the spread diverges, you can take advantage of this by going long or short on the spread.
    
To illustrate, imagine you have a long position in AAPL worth \\\\$2000 and a short position in IBM worth \\\\$2000. This gives you a net spread of \\\\$0. Since you expected AAPL and IBM to move together, then if the spread increases significantly above \\\\$0, you would short the spread in the expectation that it will return to \\\\$0, it's natural equilibrium. Similarly, if the value drops significantly below \\\\$0, you would long the spread and capture the profits as its value returns to \\\\$0. In our application, the Kalman filter will be used to track the hedging ratio between our equities to ensure that the portfolio value is stationary, which means it will continue to exhibit mean-reversion behavior.

In [7]:
// QuantBook C# Research Environment
// For more information see https://www.quantconnect.com/docs/research/overview
#load "../QuantConnect.csx"
var qb = new QuantBook();
var via = qb.AddEquity("VIA", Resolution.Daily).Symbol;
var viab = qb.AddEquity("VIAB", Resolution.Daily).Symbol;

In [8]:
public class KalmanFilter
{
    
    private decimal delta = 0.0001M;
    private decimal[,] wt = new decimal[,] {{0.0001M / (1 - 0.0001M), 0}, {0 , 0.0001M / (1 - 0.0001M)}};
    private decimal vt = 0.001M;
    private decimal[] theta = new decimal[2] {0, 0};
    private decimal[,] P = new decimal[,] {{0, 0}, {0, 0}};
    private decimal[,] R = null;
    private decimal qty = 2000;
    private decimal[,] C = new decimal[2,2];
   
    
    public List<decimal> Update(decimal priceOne, decimal priceTwo)
    {
//      Create the observation matrix of the latest prices
//      of TLT and the intercept value (1.0)
        decimal[] F = new decimal[2] {priceOne, 1.0M};
        decimal y = priceTwo;
        
        
//      The prior value of the states \theta_t is
//      distributed as a multivariate Gaussian with
//      mean a_t and variance-covariance R_t
        if(R == null){
            R = new decimal[,] {{0, 0}, {0, 0}};
        }else{           
            for(int k = 0; k <= 1; k++){
                for(int j = 0; j <= 1; j++){
                    R[k,j] = C[k,j] + wt[k,j];
                }
            }
        }
        
//      Calculate the Kalman Filter update
//      ----------------------------------
//      Calculate prediction of new observation
//      as well as forecast error of that prediction
        decimal yhat = F[0] * theta[0] + F[1] * theta[1];
        decimal et = y - yhat;
        
//      Q_t is the variance of the prediction of
//      observations and hence \sqrt{Q_t} is the
//      standard deviation of the predictions
        decimal[] FR = new decimal[2];
        FR[0] = F[0] * R[0,0] + F[0] * R[0,1];
        FR[1] = F[1] * R[1,0] + F[1] * R[1,1];
        
        decimal Qt = FR[0] * F[0] + FR[1] * F[1] + vt;
        decimal sqrtQt  = (decimal)Math.Sqrt(Convert.ToDouble(Qt));

        
//      The posterior value of the states \theta_t is
//      distributed as a multivariate Gaussian with mean
//      m_t and variance-covariance C_t
        decimal[] At = new decimal[2] {FR[0] / Qt, FR[1] / Qt};
        theta[0] = theta[0] + At[0] * et;
        theta[1] = theta[1] + At[1] * et;
        C[0,0] = R[0,0] - At[0] * FR[0];
        C[0,1] = R[0,1] - At[0] * FR[1];
        C[1,0] = R[1,0] - At[1] * FR[0];
        C[1,1] = R[1,1] - At[1] * FR[1];
        decimal hedgeQuantity = Math.Floor(qty * theta[0]);
        
        return new List<decimal>() {et, sqrtQt, hedgeQuantity};        
    }    
    
  

}

Now, we initialize the Kalman Filter, grab our data, and then run the Kalman Filter update process over the data.

In an algorithm, the kf.qty variable is the number of shares to invested in VIAB, and hedge_quantity is the amount to trade in the opposite direction for VIA

In [9]:
var kf = new KalmanFilter();


var history = qb.History(qb.Securities.Keys, new DateTime(2019, 1, 1), new DateTime(2019, 1, 11), Resolution.Daily);
var prices = new Dictionary<Symbol, List<decimal>>();

foreach(var slice in history){
   
    foreach(var symbol in slice.Keys){        
        if(!prices.ContainsKey(symbol)){
            prices.Add(symbol, new List<decimal>());
        }
        prices[symbol].Add(slice.Bars[symbol].Close);
    }
}

for(var k=0; k < prices[via].Count(); k++){  
    var via_price = prices[via][k];
    var viab_price = prices[viab][k];
    var result = kf.Update(via_price, viab_price);
    Console.WriteLine(result[0] + " : " + result[1] + " : " + result[2]);
}